In [3]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

In [26]:
import pandas as pd
train=pd.read_csv("train2000.csv")
train

,Unnamed: 0,index,question,context,text,answer_start,c_id
0,0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,1,56be88473aeaaa14008c9080,"In her music, what are some recurring elements...","A self-described ""modern-day feminist"", Beyonc...","love, relationships, and monogamy",104.0,2
2,2,56be892d3aeaaa14008c908c,Where did Beyonce get her name from?,"Beyoncé Giselle Knowles was born in Houston, T...",her mother's maiden name,204.0,3
3,3,56bf74d53aeaaa14008c965a,Beyonce's mother worked in what industry?,"Beyoncé Giselle Knowles was born in Houston, T...",hairdresser and salon owner,101.0,3
4,4,56bf76ef3aeaaa14008c9664,Which of her teachers discovered Beyonce's mus...,Beyoncé attended St. Mary's Elementary School ...,dance instructor Darlette Johnson,148.0,4
...,...,...,...,...,...,...,...
1995,1995,56e1039be3433e1400422aa6,"Prior to 1917, what church was in a similar si...",The law of the Eastern Catholic Churches in fu...,the Latin or Western Church,103.0,2824
1996,1996,56e1039be3433e1400422aa7,What was different about the Eastern Churches ...,The law of the Eastern Catholic Churches in fu...,more diversity in legislation,149.0,2824
1997,1997,56e104b7e3433e1400422acb,What are the constituents of the Pēdálion?,The Greek-speaking Orthodox have collected can...,canons and commentaries upon them,43.0,2826
1998,1998,56e10f57cd28a01900c674ff,In what institution do church courts still hav...,"In the Church of England, the ecclesiastical c...",the Church of England,3.0,2827


In [27]:
ques=train.question   # all questions
from nltk.tokenize import word_tokenize
tokenized_ques = [word_tokenize(i) for i in ques] #word tokenization of all questions
print(len(tokenized_ques))

2000


In [28]:
paragra= train.context.unique()  # all unique paragraphs
from nltk.tokenize import word_tokenize
tokenized_para = [word_tokenize(i) for i in paragra] #word tokenization of all paragraphs
print(len(tokenized_para))

1238


In [29]:
paraorigi=train.context # all paragraphs
tokenized_paraoriginal = [word_tokenize(i) for i in paraorigi]
print(len(tokenized_paraoriginal))

2000


In [30]:
# function to make sentence vectors from word vectors
import numpy as np
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model[w]
            else:
                sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
    
    return np.asarray(sent_vec) / numw

In [31]:
paragraph=[]  # list of all unique paragraph vectors

for sentence in tokenized_para:
    paragraph.append(sent_vectorizer(sentence, model)) 

In [32]:
question=[]  # list of all question vectors
for sentence in tokenized_ques:
    question.append(sent_vectorizer(sentence, model)) 

In [33]:
paraoriginal=[] # list of all paragraph vectors
for sentence in tokenized_paraoriginal:
    paraoriginal.append(sent_vectorizer(sentence, model))

In [34]:
import math
def cosineValue(v1,v2):                  #compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

In [35]:
from tqdm import tqdm

In [16]:
# function  to find the accuracy of getting the correct paragraph out of the 3 and top 5 matched paragraphs 
count5=0   #  for top 5
count3=0   #  for top 3
for j in tqdm(range(len(question))):    #iterate over questions  # no. of questions=2000
    ques1=[]
    for i in range(len(paragraph)):     # iterate over paragraphs # no. of paragraph=1238
         ques1.append(cosineValue (question[j], paragraph[i]))  # storing cosine value for a particular question with each paragraph
    res = sorted(range(len(ques1)), key = lambda sub: ques1[sub])[-5:] #to store paragraph indices for top 5 cosine values
    arr0=paragraph[res[0]]  # corresponding 5 paragraph vectors
    arr1=paragraph[res[1]]
    arr2=paragraph[res[2]]
    arr3=paragraph[res[3]]
    arr4=paragraph[res[4]]
    
    para=paraoriginal[j]  # original paragraph vector (paragraph vector corresponding to the question)

    check0=para==arr0 #check whether the original vector is equal to one of the 5 vectors
    check1=para==arr1
    check2=para==arr2
    check3=para==arr3
    check4=para==arr4
    if(check2.all() or check3.all() or check4.all() ): # for top 3
        count3+=1
    if(check0.all() or check1.all() or check2.all() or check3.all() or check4.all() ): # for top 5
        count5+=1
    
print("\n")
print((count3/len(question))*100)
print((count5/len(question))*100)

100%|██████████| 2000/2000 [33:14<00:00,  1.00it/s]




56.85
64.55


In [36]:
from nltk.corpus import stopwords 
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [37]:
# to make new vectors of questions by removing stopwords 
for i in range(len(ques)):
    ques[i]=ques[i].lower()         # convert to lower case
tokenized_ques = [word_tokenize(i) for i in ques]
ques_new=[]
for i in range(len(ques)):
    filtered_sentence = [w for w in tokenized_ques[i] if not w in stop_words]  # storing words which are not stopwords

    filtered_sentence = [] 

    for w in tokenized_ques[i]: 
        if w not in stop_words: 
            filtered_sentence.append(w)   
    ques_new.append(filtered_sentence)
question_new=[]                         #  list of all new question vectors
for sentence in ques_new:
    question_new.append(sent_vectorizer(sentence, model))

C:\Users\Anujkumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
# same for all unique paragraph - removing stopwords
for i in range(len(paragra)):
    paragra[i]=paragra[i].lower()
tokenized_para = [word_tokenize(i) for i in paragra]
paragra_new=[]
for i in range(len(paragra)):
    filtered_sentence = [w for w in tokenized_para[i] if not w in stop_words] 

    filtered_sentence = [] 

    for w in tokenized_para[i]: 
        if w not in stop_words: 
            filtered_sentence.append(w)   
    paragra_new.append(filtered_sentence)
paragraph_new=[]                            #  list of all new unique paragraph vectors
for sentence in paragra_new:
    paragraph_new.append(sent_vectorizer(sentence, model))

In [39]:
#same for all paragraphs - removing stopwords
for i in range(len(paraorigi)):
    paraorigi[i]=paraorigi[i].lower()
tokenized_paraoriginal = [word_tokenize(i) for i in paraorigi]
paragraorigi_new=[]
for i in range(len(paraoriginal)):
    filtered_sentence = [w for w in tokenized_paraoriginal[i] if not w in stop_words] 

    filtered_sentence = [] 

    for w in tokenized_paraoriginal[i]: 
        if w not in stop_words: 
            filtered_sentence.append(w)   
    paragraorigi_new.append(filtered_sentence)
paraoriginal_new=[]                               #  list of all new paragraph vectors
for sentence in paragraorigi_new:
    paraoriginal_new.append(sent_vectorizer(sentence, model))

C:\Users\Anujkumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
# same function  to find the accuracy but with questions and paragraphs after removing stopwords
count3=0
count5=0
for j in tqdm(range(len(question_new))):
    ques1=[]
    for i in range(len(paragraph_new)):
         ques1.append(cosineValue (question_new[j], paragraph_new[i]))
    res = sorted(range(len(ques1)), key = lambda sub: ques1[sub])[-5:] 
    arr0=paragraph_new[res[0]]
    arr1=paragraph_new[res[1]]
    arr2=paragraph_new[res[2]]
    arr3=paragraph_new[res[3]]
    arr4=paragraph_new[res[4]]
    para=paraoriginal_new[j]

    check0=para==arr0
    check1=para==arr1
    check2=para==arr2
    check3=para==arr3
    check4=para==arr4

    if(check2.all() or check3.all() or check4.all() ): # for top 3
        count3+=1
    if(check0.all() or check1.all() or check2.all() or check3.all() or check4.all() ): # for top 5
        count5+=1
    
print("\n")
print((count3/len(question))*100)
print((count5/len(question))*100)

100%|██████████| 2000/2000 [17:32<00:00,  1.90it/s]




61.25000000000001
68.8


In [40]:
import nltk
# for each paragraph, breaking the paragraph into sentences and storing the vector of each sentence   (after removing stopwords)
paravectors=[] # contains list of parasentence_new[i] which is containing list of sentence vectors of the particular paragraph
for i in tqdm(range(len(paragra))):
    text=paragra[i]
    a_list1 = nltk.tokenize.sent_tokenize(text)  # breaking the paragraph to sentences
    tokenized_sents3 = [word_tokenize(j) for j in a_list1]
    parasent=[]            
    for i in range(len(tokenized_sents3)):
        filtered_sentence = [w for w in tokenized_sents3[i] if not w in stop_words] 
        filtered_sentence = [] 
        for w in tokenized_sents3[i]: 
            if w not in stop_words: 
                filtered_sentence.append(w)   
        parasent.append(filtered_sentence)    # storing each sentence after removing stopwords
    parasentence_new=[]                    
    for sentence in parasent:
        parasentence_new.append(sent_vectorizer(sentence, model)) # contains list of sentence vectors of the particular paragraph
    paravectors.append(parasentence_new)

100%|██████████| 1238/1238 [00:03<00:00, 340.90it/s]


In [41]:
# function which finds the accuracy of the most similar sentence in a given paragraph with a given question
def cosineValue1(v1,parasentence_new):
    cossimilar=[]
    for i in range(len(parasentence_new)):
        cossimilar.append(cosineValue(v1,parasentence_new[i]))
    res1 = sorted(range(len(cossimilar)), key = lambda sub: cossimilar[sub])[-1:]
    x= cossimilar[res1[0]]
    return x

In [25]:
# function  to find the accuracy of getting the correct paragraph (after considering only the most similar sentence in it) out of the 3 and top 5 matched paragraphs (and removing stopwords)
count3=0
count5=0
for j in tqdm(range(len(question_new))):
    ques1=[]
    for i in range(len(paragraph_new)):
         ques1.append(cosineValue1 (question_new[j],paravectors[i]))  # the parameters are the given question vector and list containing sentence vectors of a given paragraph
    res = sorted(range(len(ques1)), key = lambda sub: ques1[sub])[-5:] 
    arr0=paragraph_new[res[0]]
    arr1=paragraph_new[res[1]]
    arr2=paragraph_new[res[2]]
    arr3=paragraph_new[res[3]]
    arr4=paragraph_new[res[4]]
    para_j=paraoriginal_new[j]

    check0=para_j==arr0
    check1=para_j==arr1
    check2=para_j==arr2
    check3=para_j==arr3
    check4=para_j==arr4
    
    if(check2.all() or check3.all() or check4.all() ): # for top 3
        count3+=1
    if(check0.all() or check1.all() or check2.all() or check3.all() or check4.all() ): # for top 5
        count5+=1
    
print("\n")
print((count3/len(question))*100)
print((count5/len(question))*100)

100%|██████████| 2000/2000 [1:12:54<00:00,  2.19s/it]




65.0
71.2


In [43]:
your_ques='which holiday was the terrorist bombing planned for?'

train1 = train[train['question'].str.contains(your_ques)]
train1.reset_index(inplace = True)
x= train1.loc[0]['Unnamed: 0']     #  x: index of the given question in dataframe
ques2=[]
for i in range(len(paragraph)):     # iterate over paragraphs # no. of paragraph=1238
  ques2.append(cosineValue1 (question_new[x], paravectors[i]))  # storing cosine value for a particular question with each paragraph
res = sorted(range(len(ques2)), key = lambda sub: ques2[sub])[-5:]
print("the Question is: \n")
print(your_ques)
print("\n\nTop 5 matched paragraphs are: \n")
print("1. ",paragra[res[4]])
print("\n2. ",paragra[res[3]])
print("\n3. ",paragra[res[2]])
print("\n4. ",paragra[res[1]])
print("\n5. ",paragra[res[0]])
print("\n\n Correct paragraph is :\n")
print(paraorigi[x])

the Question is: 

which holiday was the terrorist bombing planned for?


Top 5 matched paragraphs are: 

1.  following garreth mallory's promotion to m, on a mission in mexico city unofficially ordered by a posthumous message from the previous m, 007 james bond kills three men plotting a terrorist bombing during the day of the dead and gives chase to marco sciarra, an assassin who survived the attack. in the ensuing struggle, bond steals his ring, which is emblazoned with a stylised octopus, and then kills sciarra by kicking him out of a helicopter. upon returning to london, bond is indefinitely suspended from field duty by m, who is in the midst of a power struggle with c, the head of the privately-backed joint intelligence service, consisting of the recently merged mi5 and mi6. c campaigns for britain to form alongside 8 other countries "nine eyes ", a global surveillance and intelligence co-operation initiative between nine member states, and uses his influence to close down the '0